In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import data_processing as dp
from scipy import signal
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import os
import time
import h5py
from itertools import combinations

In [ ]:
for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

In [ ]:
path2 = '.\data\正常\G11_Walking_trial_4_emg.csv'

In [260]:
Data = pd.read_csv('./processed data/featurePcwtf_W256_S64_WS32_DWTLmax_dropna_nm.csv')

# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
drop = 'P379_M050_2_OFF_A_FoG_trial_2_emg.csv'
drop2= 'P812_M050_2_B_FoG_trial_2_emg.csv'
ind_drop = (df.columns!=drop) | (df.columns!=drop2)
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

files = np.concatenate([np.array(df.columns[ind_drop]),np.array(df3.loc[:,0])])
#ind = Data.File.isin(files)
ind = (Data.File != drop) & (Data.File != drop2)
Data_sel = Data[ind]
Data_rest = Data[~ind]
#ind2 = Data_rest.File == drop
#Data_rest = Data_rest[ind2]
#Data_rest

In [261]:
feature_col = ['_IEMG','_MAV','_SSI','_VAR','_RMS',
               '_WL','_ZC','_SSC','_WAMP','_skew','_Acti','_AR','_HIST','_MDF']

feature_all = Data_sel.iloc[:,1:-1]
#ind_temp1 = feature_all.columns.str.contains('_mDWT')
#ind_temp2 = feature_all.columns.str.contains('_Coe')
#ind_temp3 = feature_all.columns.str.contains('_Scale')
#ind_temp4 = feature_all.columns.str.contains('_HIST')
#ind_temp = ind_temp1|ind_temp2|ind_temp3|ind_temp4
#ind_temp = feature_all.columns.str.contains('mDWT')
#feature = feature_all.loc[:,~ind_temp]
feature = Data_sel.iloc[:,1:-1]
#feature = Data_sel.iloc[:,:-2]
y = Data_sel.Label
#feature2_all = Data_rest.iloc[:,1:-1]
#feature2 = feature2_all.loc[:,ind_temp]
feature2 = Data_rest.iloc[:,1:-1]
#feature2 = Data_rest.iloc[:,:-2]
y2 = Data_rest.Label

In [ ]:
path = './processed data/data_set_after_window_withoutSC.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]

In [ ]:
import imp
imp.reload(dp)

In [ ]:
import ctypes
player = ctypes.windll.kernel32
player.Beep(1000,200)

In [91]:
feature2

,LEFT_TA_IEMG,LEFT_TS_IEMG,LEFT_BF_IEMG,LEFT_RF_IEMG,RIGHT_TA_IEMG,RIGHT_TS_IEMG,RIGHT_BF_IEMG,RIGHT_RF_IEMG,LEFT_TA_SSI,LEFT_TS_SSI,...,RIGHT_TS_Mean_Scale,RIGHT_TS_Median_Scale,RIGHT_BF_Mean_Coe,RIGHT_BF_Min_Coe,RIGHT_BF_Mean_Scale,RIGHT_BF_Median_Scale,RIGHT_RF_Mean_Coe,RIGHT_RF_Min_Coe,RIGHT_RF_Mean_Scale,RIGHT_RF_Median_Scale
10948,47.382390,46.594215,35.291904,116.145730,45.856050,123.335860,31.193258,51.369167,15.538249,18.513243,...,13.632590,10.0,0.177132,0.117279,17.253480,16.0,0.264891,0.133763,18.165906,17.0
10949,44.828053,39.516617,34.159218,113.909065,42.923042,128.338090,31.143126,56.215000,14.999877,14.308270,...,14.116727,11.0,0.182103,0.117114,17.220130,16.0,0.224715,0.147266,17.052881,15.0
10950,43.025867,37.990456,29.838589,114.139595,42.155900,129.607850,30.133968,55.165016,14.461066,13.801164,...,14.749943,12.0,0.158897,0.114445,16.446790,15.0,0.260783,0.131384,18.002591,17.0
10951,44.651604,40.892870,31.411098,104.653760,45.591217,132.558000,31.221636,56.788517,15.174104,14.831537,...,14.029997,11.0,0.176271,0.121978,16.845654,15.0,0.326726,0.137582,19.130992,19.0
10952,43.412860,39.094160,30.660444,107.645220,47.048786,133.734630,29.432373,50.962555,14.925791,14.161764,...,14.439348,12.0,0.150912,0.114144,16.054218,14.0,0.272874,0.127858,16.135058,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16101,70.580040,75.913590,126.304886,28.200338,86.091930,38.523070,25.148800,41.124783,41.334454,46.245580,...,15.942846,14.0,0.187388,0.081706,16.524947,14.0,0.272937,0.102111,16.223742,14.0
16102,76.214836,73.919020,130.151630,27.541174,83.605705,39.387753,23.524101,32.228546,47.416060,45.405724,...,16.790883,15.0,0.182730,0.075469,16.943298,15.0,0.197590,0.086402,16.462787,14.0
16103,55.280030,103.922540,128.093640,27.076658,83.045140,33.136265,23.160860,29.875586,27.816164,73.014786,...,17.634540,16.0,0.191178,0.080705,17.288791,16.0,0.167673,0.075130,14.920290,12.0
16104,78.838950,81.898060,130.907060,30.604612,73.035170,33.454340,23.480703,28.396082,48.587643,54.006077,...,18.773659,18.0,0.168605,0.086141,17.410496,16.0,0.176414,0.073928,16.442348,14.0


In [ ]:
width = 256
threshold_WAMP = 30
threshold_ZC = 0
threshold_SSC = 1
bins=9
bound = 70
HIST_range = (-bound,bound)

In [ ]:
path = './processed data/data_set_after_window_S64_withoutSC_allPa.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]
feature = dp.generate_feature(x,threshold_WAMP=threshold_WAMP,
                              threshold_ZC=threshold_ZC,
                              threshold_SSC=threshold_SSC,
                              bins=bins,ranges=HIST_range)
#feature2 = dp.generate_feature(x2)
player.Beep(1000,200)

In [ ]:
with h5py.File('./processed data/nfeatures_W256_S64_WAMP30.hdf5','r') as f:
    feature = f['features'][...]
    y = f['labels'][...]

In [ ]:
feature2,y2 = dp.pipeline_feature(path2,width=256,stride=64,
                                  scaler=False,
                                  threshold_WAMP=threshold_WAMP,
                                  threshold_ZC=threshold_ZC,
                                  threshold_SSC=threshold_SSC,
                                  bins=bins,ranges=HIST_range)

In [ ]:
with h5py.File('./processed data/nfeatures_rest_W256_S64.hdf5','r') as f:
    feature2 = f['features'][...]
    y2 = f['labels'][...]

In [ ]:
sc = StandardScaler(with_mean=True)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.transform(feature2)

# Model

In [8]:
import tensorflow as tf
from tensorflow.keras import layers,Model,callbacks,regularizers
from sklearn.model_selection import train_test_split,cross_validate
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE

In [6]:
tf.executing_eagerly()

True

In [278]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y.copy()
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=345,
                                                   shuffle=True)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.25,
                                                       random_state=555,
                                                       shuffle=True)
    
    #sm = BorderlineSMOTE(random_state=50,kind='borderline-1')
    #sm = SMOTE(random_state=50)
    #print(y_full.shape)
    #x_full,y_full = sm.fit_resample(x_full,y_full)
    #print(y_full_n.shape)
    sc = StandardScaler(with_mean=True)
    x_train = sc.fit_transform(x_full)
    #pca = PCA(n_components=200)
    #x_train = pca.fit_transform(x_train)
    #x_valid = sc.transform(x_valid)
    x_test = sc.transform(x_test)
    #x_test = pca.transform(x_test)
    #x_train = x_full
    
    early_stopping = callbacks.EarlyStopping(patience = 10,
                                             monitor = 'val_accuracy', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_full,validation_data=[x_test,y_test],
                        epochs=200,batch_size=32,class_weight=cw,
                        callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(y_valid,np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = metrics.accuracy_score(y_full_n,y_pred_ta>0.5)
        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full_n,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))

    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    #print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,test,sc

def test_model(model,feature,y,sc):
    ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
    #ind = ((y==0)|(y==1)|(y==2)|(y==6))
    y_01 = y.copy()
    y_01[ind] = 1
    #print(set(y_01))
    oh_ec = OneHotEncoder()
    y_01 = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
    #print(y_01)
    y_pred=model.predict(sc.transform(feature))
    test = metrics.accuracy_score(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1))
    #test = metrics.accuracy_score(y_01,y_pred>0.5)
    
    print('acc:%f'%test)
    print(metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1)))
    #print(metrics.confusion_matrix(y_01,y_pred>0.5))
    return test

def sparse_cost_sensitive_loss (y_true,y_pred):
    #cost_matrix = tf.constant([[0,1.,1,1.],
    #              [2,0,5,5],
    #              [1,1,0,1],
    #              [1.,2.,1,0]])
    #cost_matrix = tf.constant([[0,5.,1],
    #              [1,0,1],
    #              [1.5,2.,0]])
    cost_matrix = tf.constant([[0,2.],
                  [5.,0]])
    batch_cost_matrix = tf.nn.embedding_lookup(cost_matrix, tf.argmax(y_true,axis=1))
    eps = 1e-6
    probability = tf.clip_by_value(y_pred, eps, 1-eps)
    cost_values = tf.math.log(1-probability)*batch_cost_matrix
    loss = tf.reduce_mean(-tf.reduce_sum(cost_values, axis=1))
    return loss

## ANN

In [234]:
ind = ((y==1)|(y==2)|(y==3)|(y==6))
#ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
ind_f = [0,1,6,42,46,57,62]
#y_01 = y[ind].copy()
#y_01[y_01==1]=0
#y_01[y_01==2]=1
#y_01[y_01==3]=2
#y_01[y_01==6]=3
y_01 = y[ind].copy()
#y_01[ind] = 1
oh_ec = OneHotEncoder()
y_oh = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
x_full,x_test,y_full,y_test = train_test_split(feature.loc[ind,:],y_01,test_size=0.2,random_state=123,shuffle=False)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555,shuffle=True)

In [279]:
input_ = layers.Input(shape=feature.shape[1:])
l1 = layers.Dense(128,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(drop1)
drop2 = layers.Dropout(0.2)(l2)
l3 = layers.Dense(32,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(drop2)
drop3 = layers.Dropout(0.2)(l3)
l4 = layers.Dense(16,activation='elu')(drop3)
drop4 = layers.Dropout(0.2)(l4)
#l5 = layers.Dense(8,activation='relu')(drop4)
#drop5 = layers.Dropout(0.5)(l5)
output = layers.Dense(2,activation='softmax')(drop4)
model = Model(inputs=[input_],outputs=[output])

In [280]:
model.compile(loss=sparse_cost_sensitive_loss,optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=200,batch_size=50,
                   callbacks=[early_stopping])

# Train model

In [281]:
#sc = StandardScaler(with_mean=True)
#feature_sc = sc.fit_transform(feature)
#feature2_sc = sc.transform(feature2)
#pca = PCA(n_components=160,copy=True)
#feature_pca = pca.fit_transform(feature)
#feature2_pca = pca.transform(feature2)
train,test,sc = train_model(model,feature,np.array(y),True)
#rest = test_model(model,feature2,np.array(y2),sc)
#acc = [train,valid,test,rest]

Train on 66288 samples, validate on 16573 samples
Epoch 1/200
66288/66288 [==============================] - 7s 103us/sample - loss: 0.4466 - accuracy: 0.9404 - val_loss: 0.2340 - val_accuracy: 0.9690
Epoch 2/200
66288/66288 [==============================] - 6s 93us/sample - loss: 0.2736 - accuracy: 0.9650 - val_loss: 0.1943 - val_accuracy: 0.9736
Epoch 3/200
66288/66288 [==============================] - 6s 87us/sample - loss: 0.2347 - accuracy: 0.9693 - val_loss: 0.1767 - val_accuracy: 0.9764
Epoch 4/200
66288/66288 [==============================] - 6s 86us/sample - loss: 0.2087 - accuracy: 0.9726 - val_loss: 0.1600 - val_accuracy: 0.9777
Epoch 5/200
66288/66288 [==============================] - 6s 86us/sample - loss: 0.1946 - accuracy: 0.9741 - val_loss: 0.1602 - val_accuracy: 0.9770
Epoch 6/200
66288/66288 [==============================] - 6s 95us/sample - loss: 0.1794 - accuracy: 0.9762 - val_loss: 0.1543 - val_accuracy: 0.9774
Epoch 7/200
66288/66288 [========================

Epoch 55/200
66288/66288 [==============================] - 6s 90us/sample - loss: 0.0392 - accuracy: 0.9954 - val_loss: 0.1117 - val_accuracy: 0.9902
Epoch 56/200
66288/66288 [==============================] - 6s 86us/sample - loss: 0.0368 - accuracy: 0.9952 - val_loss: 0.1217 - val_accuracy: 0.9900
train: 
 [[55390   115]
 [    0 10783]]
test: 
 [[13714   110]
 [   47  2702]]
test:0.990527
train:0.998265


In [282]:
test_model(model,np.array(feature2),np.array(y2),sc)

acc:0.864530
[[535 115]
 [ 71 652]]


0.864530225782957

In [230]:
pd.Series(y2).value_counts()

0    460
2    430
1     33
3      8
Name: Label, dtype: int64

In [ ]:
#test_model(model,feature2_sc,np.array(y2))
ind = ((y==1)|(y==2)|(y==6))
y_01 = y[ind].copy()
oc = OneHotEncoder()
y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
pred=model.predict(feature_sc[ind,:])
np.argmax(pred,axis=1)
np.argmax(y_01,axis=1)
metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(pred,axis=1))

In [ ]:
#acc_ann = pd.DataFrame(acc,columns=['dnn1'],index=['train','valid','test','rest data'])
#acc_ann_com=pd.concat([acc_ann_com,acc_ann.T])
#acc_ann.loc['drop_mDWT',:]=[train,valid,test,rest]
acc_ann_com.loc['dnn2',:]=acc
acc_ann_com

# test feature from Left or Right

In [ ]:
input_ = layers.Input(shape=feature.loc[:,ind_temp].shape[1:])
l1 = layers.Dense(128,activation='elu')(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='elu')(drop1)
drop2 = layers.Dropout(0.2)(l2)
#l3 = layers.Dense(32,activation='elu')(drop2)
#drop3 = layers.Dropout(0.2)(l3)
#l4 = layers.Dense(16,activation='selu')(l3)
#drop4 = layers.Dropout(0.2)(l4)
output = layers.Dense(1,activation='sigmoid')(drop2)
model = Model(inputs=[input_],outputs=[output])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [209]:
acc={}
cols = ['LEFT','RIGHT']

#sc = StandardScaler(with_mean=True)

for col in cols:
    ind_temp=feature.columns.str.contains(col)
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    #acc[col] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 23125 samples, validate on 5782 samples
Epoch 1/200
23125/23125 [==============================] - 4s 167us/sample - loss: 1.9657 - accuracy: 0.5308 - val_loss: 1.3537 - val_accuracy: 0.6404
Epoch 2/200
23125/23125 [==============================] - 3s 109us/sample - loss: 1.3913 - accuracy: 0.6479 - val_loss: 1.1555 - val_accuracy: 0.7008
Epoch 3/200
23125/23125 [==============================] - 3s 110us/sample - loss: 1.2045 - accuracy: 0.7052 - val_loss: 1.0439 - val_accuracy: 0.7771
Epoch 4/200
23125/23125 [==============================] - 2s 105us/sample - loss: 1.1076 - accuracy: 0.7394 - val_loss: 0.9467 - val_accuracy: 0.7608
Epoch 5/200
23125/23125 [==============================] - 2s 86us/sample - loss: 1.0526 - accuracy: 0.7501 - val_loss: 0.9212 - val_accuracy: 0.7508
Epoch 6/200
23125/23125 [==============================] - 2s 93us/sample - loss: 0.9990 - accuracy: 0.7677 - val_loss: 0.8937 - val_accuracy: 0.7975
Epoch 7/200
23125/23125 [======================

23125/23125 [==============================] - 2s 71us/sample - loss: 0.8517 - accuracy: 0.8045 - val_loss: 0.8904 - val_accuracy: 0.8210
Epoch 9/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.8091 - accuracy: 0.8131 - val_loss: 1.0212 - val_accuracy: 0.8426
Epoch 10/200
23125/23125 [==============================] - 2s 72us/sample - loss: 0.7750 - accuracy: 0.8260 - val_loss: 1.0699 - val_accuracy: 0.8438
Epoch 11/200
23125/23125 [==============================] - 2s 72us/sample - loss: 0.7591 - accuracy: 0.8365 - val_loss: 0.9963 - val_accuracy: 0.8513
Epoch 12/200
23125/23125 [==============================] - 2s 70us/sample - loss: 0.7071 - accuracy: 0.8416 - val_loss: 1.0759 - val_accuracy: 0.8464
Epoch 13/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.6955 - accuracy: 0.8444 - val_loss: 1.0100 - val_accuracy: 0.8442
Epoch 14/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.7017 - accuracy: 0.8478

In [ ]:
acc_lr=pd.DataFrame(acc,index=['train','valid','test','rest data'])#.to_csv('./results/acc_lr_ann.csv')
#acc_com=pd.concat([acc_ann_com.drop(['dnn','dnn1'],'index'),acc_lr.T])
acc_com

# test features from 2 of 8 signals

In [ ]:
#test features from 2 of 8 signals

acc_2f={}
cols = [ 'LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF',
        'RIGHT_TA','RIGHT_TS','RIGHT_BF', 'RIGHT_RF']


for p in combinations(cols[:4],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)
    
for p in combinations(cols[4:],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)

In [ ]:
#acc_2f_ann = pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T
#acc_com = pd.concat([acc_com,acc_2f_ann])
acc_com.to_csv('./results/dropna/acc_com_ann.csv')

In [ ]:
pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T

# Test on some of rest

In [ ]:
some_of_rest = ['正常/P940_MSham_B_Walking_trial_6_emg.csv',
                '正常/P940_M050_B_Walking_trial_4_emg.csv',
                '正常/P812_M100_A_Walking_trial_3_emg.csv',
                '正常/P645_M050_A_Walking_trial_3_emg.csv',
                '正常/P623_Msham_B_Walking_trial_2_emg.csv',
                '正常/P551_M50_B_Walking_trial_6_emg.csv',
                'P379_M050_2_OFF_A_FoG_trial_1_emg.csv',
                'P551_M050_2_B_FoG_trial_2_emg.csv']
#booster = xgb.Booster()
#booster.load_model('./model/XGBoost_W256_S64_Left.json')
#model = xgb.XGBClassifier()
#model._Booster = booster
acc = []
columns=['LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF']
for file in some_of_rest:
    path = './data/'+file
    feature2,y2 = dp.pipeline_feature(path,width=256,stride=64,scaler=False,
                                      threshold_WAMP=threshold_WAMP,
                                      threshold_ZC=threshold_ZC,
                                      threshold_SSC=threshold_SSC,
                                      bins=bins,
                                      ranges=HIST_range,
                                      show_para=False,
                                      filt = 250)
    feature2_sc = sc.transform(feature2)
    acc += [test_model(model,feature2_sc,y2)]
print(acc)

In [ ]:
y_pred=model.predict(feature2_sc)
metrics.accuracy_score(y2,y_pred>0.5)

## RNN

In [ ]:
feature_re = feature_sc.reshape((-1,feature.shape[1],1))
x_full,x_test,y_full,y_test = train_test_split(np.array(feature_re),np.array(y_01),test_size=0.2,random_state=123)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555)

In [ ]:
input_ = layers.Input(shape=[feature.shape[1],1])
lstm1 = layers.GRU(100)(input_)
#lstm2 = layers.LSTM(20)(lstm1)
output = layers.Dense(1,activation='sigmoid')(lstm1)
model = Model(inputs=[input_],outputs=[output])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=100,batch_size=500,
                   callbacks=[early_stopping])

In [ ]:
feature2_re = feature2_sc.reshape((-1,feature.shape[1],1))

In [ ]:
test_model(model,x_test,y_test)

In [104]:
cost_matrix=np.array([[0,1,1,1],
                  [1,0,1,1],
                  [10,100,0,10],
                  [1.,1.,1,0]])

In [105]:
t = tf.constant(cost_matrix)

In [127]:
(t@tf.transpose(t)).numpy()

array([[3.00e+00, 2.00e+00, 1.10e+02, 2.00e+00],
       [2.00e+00, 3.00e+00, 2.00e+01, 2.00e+00],
       [1.10e+02, 2.00e+01, 1.02e+04, 1.10e+02],
       [2.00e+00, 2.00e+00, 1.10e+02, 3.00e+00]])

In [112]:
tf.transpose(t)

<tf.Tensor: id=1567100, shape=(4, 4), dtype=float64, numpy=
array([[  0.,   1.,  10.,   1.],
       [  1.,   0., 100.,   1.],
       [  1.,   1.,   0.,   1.],
       [  1.,   1.,  10.,   0.]])>